In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown

### Refresh dot env

In [2]:
load_dotenv(override=True)

True

In [3]:
open_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

### Create initial query to get challange reccomendation

In [4]:
query = 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. '
query += 'Answer only with the question, no explanation.'

messages = [{'role':'user', 'content':query}]

In [5]:
print(messages)

[{'role': 'user', 'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]


### Call openai gpt-4o-mini 

In [6]:
openai = OpenAI()

response = openai.chat.completions.create(
    messages=messages,
    model='gpt-4o-mini'
)

challange = response.choices[0].message.content


In [7]:
print(challange)

How would you approach resolving a complex ethical dilemma that involves competing rights, where the interests of one marginalized group conflict with those of another, and what framework would you use to justify your decision?


In [8]:
competitors = []
answers = []

### Create messages with the challange query

In [9]:
messages = [{'role':'user', 'content':challange}]

In [10]:
print(messages)

[{'role': 'user', 'content': 'How would you approach resolving a complex ethical dilemma that involves competing rights, where the interests of one marginalized group conflict with those of another, and what framework would you use to justify your decision?'}]


In [11]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [12]:
from threading import Thread

In [13]:
def gpt_mini_processor():
    modleName = 'gpt-4o-mini'
    competitors.append(modleName)
    response_gpt = openai.chat.completions.create(
        messages=messages,
        model=modleName
    )
    answers.append(response_gpt.choices[0].message.content)

def gemini_processor():
    gemini = OpenAI(api_key=google_api_key, base_url='https://generativelanguage.googleapis.com/v1beta/openai/')
    modleName = 'gemini-2.0-flash'
    competitors.append(modleName)
    response_gemini = gemini.chat.completions.create(
        messages=messages,
        model=modleName
    )
    answers.append(response_gemini.choices[0].message.content)

def llama_processor():
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    modleName = 'llama3.2'
    competitors.append(modleName)
    response_llama = ollama.chat.completions.create(
        messages=messages,
        model=modleName
    )
    answers.append(response_llama.choices[0].message.content)

### Paraller execution of LLM calls

In [14]:
thread1 = Thread(target=gpt_mini_processor)
thread2 = Thread(target=gemini_processor)
thread3 = Thread(target=llama_processor)

thread1.start()
thread2.start()
thread3.start()

thread1.join()
thread2.join()
thread3.join()

In [15]:
print(competitors)
print(answers)

['gpt-4o-mini', 'gemini-2.0-flash', 'llama3.2']
["Resolving a complex ethical dilemma that involves competing rights, particularly when it involves marginalized groups, requires a careful and nuanced approach. Here’s a structured way to approach such dilemmas:\n\n### 1. **Identify the Stakeholders:**\n   - Clearly define all parties involved. Understand who the marginalized groups are and their specific interests and rights.\n   - Recognize any overlapping identities that might complicate the situation, such as intersections of race, gender, socioeconomic status, etc.\n\n### 2. **Gather Information:**\n   - Collect relevant data, historical context, and any existing policies or laws that pertain to the situation.\n   - Understand the perspectives of both sides, including the historical grievances and rights claims of both marginalized groups.\n\n### 3. **Clarify the Ethical Principles Involved:**\n   - Identify the key ethical principles at play such as justice, equity, autonomy, harm 

In [16]:
for competitor, answer in zip(competitors, answers):
    print(f'Competitor:{competitor}\n\n{answer}')

Competitor:gpt-4o-mini

Resolving a complex ethical dilemma that involves competing rights, particularly when it involves marginalized groups, requires a careful and nuanced approach. Here’s a structured way to approach such dilemmas:

### 1. **Identify the Stakeholders:**
   - Clearly define all parties involved. Understand who the marginalized groups are and their specific interests and rights.
   - Recognize any overlapping identities that might complicate the situation, such as intersections of race, gender, socioeconomic status, etc.

### 2. **Gather Information:**
   - Collect relevant data, historical context, and any existing policies or laws that pertain to the situation.
   - Understand the perspectives of both sides, including the historical grievances and rights claims of both marginalized groups.

### 3. **Clarify the Ethical Principles Involved:**
   - Identify the key ethical principles at play such as justice, equity, autonomy, harm reduction, and fairness. 
   - Examin

In [17]:
together = ''
for index, answer in enumerate(answers):
    together += f'# Response from competitor {index + 1}\n\n'
    together += answer + '\n\n'

In [18]:
print(together)

# Response from competitor 1

Resolving a complex ethical dilemma that involves competing rights, particularly when it involves marginalized groups, requires a careful and nuanced approach. Here’s a structured way to approach such dilemmas:

### 1. **Identify the Stakeholders:**
   - Clearly define all parties involved. Understand who the marginalized groups are and their specific interests and rights.
   - Recognize any overlapping identities that might complicate the situation, such as intersections of race, gender, socioeconomic status, etc.

### 2. **Gather Information:**
   - Collect relevant data, historical context, and any existing policies or laws that pertain to the situation.
   - Understand the perspectives of both sides, including the historical grievances and rights claims of both marginalized groups.

### 3. **Clarify the Ethical Principles Involved:**
   - Identify the key ethical principles at play such as justice, equity, autonomy, harm reduction, and fairness. 
   - 

### Prompt to judge the LLM results

In [19]:
to_judge = f'''You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{challange}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

'''

In [20]:
to_judge_message = [{'role':'user', 'content':to_judge}]

### Execute o3-mini to analyze the LLM results

In [21]:
openai = OpenAI()
response = openai.chat.completions.create(
    messages=to_judge_message,
    model='o3-mini'
)
result = response.choices[0].message.content
print(result)

{"results": ["2", "1", "3"]}


In [22]:
results_dict = json.loads(result)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: gemini-2.0-flash
Rank 2: gpt-4o-mini
Rank 3: llama3.2
